# PawsPlace

Im Rahmen unseres Studienmoduls "Geomarketing mit Geodaten" entwickeln wir eine spezialisierte Anwendung, die darauf ausgerichtet ist, Hundehaltern bei der Wohnungssuche zu assistieren, indem sie Grünflächen in der Nähe einer eingegebenen Adresse identifiziert. Diese Anwendung verwendet Geodaten, um die Wohngebiete auf ihre Tauglichkeit für Hundebesitzer zu prüfen, speziell in Bezug auf die Verfügbarkeit und Erreichbarkeit von Grünflächen.


Als erstes werden die benötigten Imports vorgenommen. Wir werden die allgemeinen Reports im oberen Importfenster ansiedeln. Spezifische Imports für Anwendnungen, werden direkt im jeweiligen Kapitel angesiedelt

# Installationen

In [3]:
!pip install osmnx
!pip install geopandas
!pip install folium


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Simone\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Simone\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/b9/98/9ba4b9d2d07dd32765ddb4e4c189dcbdd7dca4d5a735e2e4ea756f40c36b/folium-0.16.0-py2.py3-none-any.whl.metadata
  Using cached folium-0.16.0-py2.py3-none-any.whl.metadata (3.6 kB)
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/75/ca/6074ab4a04dd1a503201c18091b3426f3709670115fae316907a97f98d75/branca-0.7.2-py3-none-any.whl.metadata
  Using cached branca-0.7.2-py3-none-any.whl.metadata (1.5 kB)
Using cached folium-0.16.0-py2.py3-none-any.whl (100 kB)
Using cached branca-0.7.2-py3-none-any.whl (25 kB)



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Simone\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Nun sind die ersten Installationen erledigt. Als nächstes werden die verwendeten Librarys importiert und OSMNX konfiguriert

In [4]:
import osmnx as ox
import geopandas as gpd
import folium
from IPython.display import display, clear_output
 
# Konfigurieren von osmnx
ox.config(log_console=True, use_cache=True)

C:\Users\Simone\AppData\Local\Temp\ipykernel_6724\1664835571.py:7: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.config(log_console=True, use_cache=True)


# Funktion zum Laden der Daten und Anzeigen auf einer Karte

In [ ]:
def load_and_display(address):
    try:
        # Geocode Adresse
        location = ox.geocoder.geocode(address)
        
        # Find parks within a 1km radius
        tags = {'leisure': 'park'}
        gdf = ox.geometries_from_point(location, tags, dist=1000)
        
        # Create and display map
        m = folium.Map(location=[location[0], location[1]], zoom_start=14)
        folium.Marker([location[0], location[1]], popup='Ihre Adresse').add_to(m)
        
        for _, row in gdf.iterrows():
            if row['geometry'].geom_type == 'Polygon':
                folium.Polygon(
                    locations=[(lat, lon) for lon, lat in row['geometry'].exterior.coords],
                    color='green', fill=True, fill_color='green'
                ).add_to(m)
        
        display(m)
    except Exception as e:
        print(f"An error occurred: {e}")


# Schritt 3: Aufrufen der Funktion mit einer festgelegten Adresse

In [5]:
# Adresse direkt setzen und Funktion aufrufen
# load_and_display("Zürich, Switzerland")
load_and_display("Brandstrasse 1, Schlieren, Switzerland")


C:\Users\j17jo\AppData\Local\Temp\ipykernel_12892\75995015.py:8: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_point(location, tags, dist=1000)


## Anbindung PostGRESQL

In [1]:
import psycopg2


# Verbindungsdaten
host = "localhost"
dbname = "deine_datenbank"
user = "dein_user"
password = "12345678"

# Verbindung aufbauen
conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password
)

# Einen Cursor erstellen
cur = conn.cursor()

# Eine SQL-Abfrage ausführen
cur.execute("SELECT * FROM deine_tabelle LIMIT 5;")

# Daten abrufen
rows = cur.fetchall()

for row in rows:
    print(row)

# Ressourcen freigeben
cur.close()
conn.close()


ModuleNotFoundError: No module named 'psycopg2'

## Eingabe User

Geben Sie die Adresse ein, an der wir nach Grünflächen suchen sollen?

Vielen Dank für Ihre Angabe.

## Verarbeitung Daten

Nun werden die Daten verarbeitet

## Anzeige Daten

Hier wird die Map ausgegeben

Hier werden Berechnungen für Entfernungen ausgegeben

In diesem Bereich können Filter definiert werden für die Spaziergang länge

## Evaluation

Zusammenfassung der Ergebnisse